In [1]:
# hidden num remain unchange

In [1]:
import torch
old = torch.load('/home/workspace/util/surgery/titanic_demo/runs/May24_08-22-52_1d7a14f7cd78old_network with old input from scratch/oldnetwork_oldinput_from_scratch_bs32.ckpt')

In [44]:
# hidden num remain unchange
import os
import numpy as np
from torch import optim
from pretreat import *
%matplotlib inline
np.random.seed(0)
training = get_pretreated_training_data()
old_input_dict = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'cabine_n', 'IsAlone']
new_input_dict = old_input_dict + ['sp', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S']
loader = get_dataloader(training, new_input_dict, bs=32)
new_net = MLP(input_dim = len(new_input_dict), hidden_dims=[40, 50, 100, 100, 100, 100,]).cuda()

/home/miniconda/envs/tftorch/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [45]:
layer = 0
w = "layers.{}.weight".format(layer)
b = "layers.{}.bias".format(layer)
w1 = old['para'][w]
b1 = old['para'][b]

inwmap = {}
for i, f in enumerate(old['input']):
    inwmap[f] = w1[:, i]

inbmap = {} # 和hidden node 相关
for i in range(w1.shape[0]):
    inbmap[i] = b1[i]

In [46]:
# 增加了输入 和中间层, 对于多出来的进行0初始化, 假设所有多出来的node都在原来hidden的下面同一侧
for i, (name, g) in enumerate(new_net.named_parameters()):
    old_g = old['para'][name].data
    if old_g.shape != g.shape:
        if i == 1:
            new_g = torch.zeros_like(g)
            for hidden_n, b in inbmap.items():
                new_g[hidden_n] = b
        elif i == 0:
            new_g = torch.zeros_like(g)
            for j, f in enumerate(new_input_dict):
                if f in inwmap.keys():
                    old_w = inwmap[f]
                    new_g[:old_w.shape[0], j] = old_w
        elif i == 2:
            new_g = g.clone().detach()
            new_g[:, :old_g.shape[1]] = old_g
        else:
            assert False
        g.data = new_g
    else:
        # print(name, g.shape)
        g.data = old['para'][name].data
    

In [47]:
new = new_net.state_dict()

In [48]:
new['layers.0.weight']

tensor([[ 8.5299e-03, -2.2137e-01,  4.2435e-01, -3.4890e-01,  1.6520e-01,
          3.0243e-02,  5.2682e-03,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-9.4769e-01,  2.8481e-02,  3.4451e-01, -6.2756e-01,  1.5863e-01,
         -4.7748e-01, -8.3875e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 4.3869e-01, -4.8007e-01, -1.1632e-01, -3.6142e-01,  4.2263e-01,
          1.9024e-01, -1.9510e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 5.6465e-01, -9.9225e-02,  8.4840e-01,  4.3230e-01, -1.4533e-02,
         -1.0773e-01,  3.3058e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.1491e-01,  2.4626e-01, -4.7582e-01, -4.1035e-01, -1.5415e-01,
         -1.4015e+00, -4.0408e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 5.100

In [49]:
# get surgery之后最初始的acc
sum_acc = 0
for x, y in loader:
    x, y = x.cuda(), y.cuda()
    outputs=new_net(x)
    _,id=torch.max(outputs.data,1)
    acc=torch.sum(id==y.data)
    sum_acc += acc.item() 
print(sum_acc)

752


/home/miniconda/envs/tftorch/lib/python3.6/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [50]:
lr = 0.001
epoches = 2000 
comment = 'hidden_add_new_net with new input Using Surgery'
model_name = 'hidden_add_newnetwork_newinput_usingSurgery.ckpt'
input_dic = new_input_dict
acc_cur, loss_cur = train(new_net, loader, comment, model_name, input_dic, )